In [1]:
from generativeAgent import GAGroup

In [2]:
# for prototype storage
import json

def storage_agent_meta(agents, city, groups=None, groups_des=None):
    stores = []
    if groups == None:
        for agent in agents:
            agent_dict = {}
            agent_dict['home'] = agent.Brain.Memory.Working.Reason['Homeplace']
            agent_dict['work'] = agent.Brain.Memory.Working.Reason['Workplace']
            agent_dict['genderDescription'] = agent.Brain.Memory.Working.Reason['genderDescription']
            agent_dict['educationDescription'] = agent.Brain.Memory.Working.Reason['educationDescription']
            agent_dict['consumptionDescription'] = agent.Brain.Memory.Working.Reason['consumptionDescription']
            agent_dict['occupationDescription'] = agent.Brain.Memory.Working.Reason['occupationDescription']
            stores.append(agent_dict)
    else:
        for group, agent_ids in groups.items():
            for aid in agent_ids:
                agent_dict = {}
                agent_dict['group'] = group
                agent_dict['home'] = agents[aid].Brain.Memory.Working.Reason['Homeplace']
                agent_dict['work'] = agents[aid].Brain.Memory.Working.Reason['Workplace']
                agent_dict['genderDescription'] = agents[aid].Brain.Memory.Working.Reason['genderDescription']
                agent_dict['educationDescription'] = agents[aid].Brain.Memory.Working.Reason['educationDescription']
                agent_dict['consumptionDescription'] = agents[aid].Brain.Memory.Working.Reason['consumptionDescription']
                agent_dict['occupationDescription'] = agents[aid].Brain.Memory.Working.Reason['occupationDescription']
                stores.append(agent_dict)
    with open(f"prototype/{city}/agents_meta.json", 'w') as f:
        json.dump(stores, f, indent=4)
    if groups_des != None:
        with open(f"prototype/{city}/protos.json", 'w') as f:
            json.dump(groups_des, f, indent=4)

In [ ]:
# change the name of target city
ag = GAGroup("./config.yaml", "GA", "san_francisco")

In [ ]:
"""
arg_1: number of agents
arg_2: Day to simulate
arg_3: (Optional) if you have already sampled profile for agents, you can use this to accelerate the initialization, other wise, it will sample profiles according to the city
"""
await ag.prepare_agents(1000, "Sunday", "prototype/san_francisco/agents_meta.json")

In [5]:
# * In-context Prototype Learning
import json
import asyncio
import copy
import random

def get_profile_message(agent):
    gender = agent.Brain.Memory.Working.Reason['genderDescription']
    educationDescription = agent.Brain.Memory.Working.Reason['educationDescription']
    consumptionDescription = agent.Brain.Memory.Working.Reason['consumptionDescription']
    occupationDescription = agent.Brain.Memory.Working.Reason['occupationDescription']
    personInfo = f"{gender} {educationDescription} {occupationDescription} {consumptionDescription}"
    return personInfo

def check_missing(agent_groups, number_of_agent):
    ids = []
    for _, value in agent_groups.items():
        for aid in value:
            if aid not in ids:
                ids.append(aid)
    ids = sorted(ids)
    gap = 0
    missing_ids = []
    for i in range(len(ids)):
        if ids[i]-gap != i:
            missing_ids.append(i)
            gap += 1
    return missing_ids

def check_repetation(agent_groups):
    ids = []
    for _, value in agent_groups.items():
        ids += value
    ids = sorted(ids)
    repet = {}
    total = 0
    for i in range(len(ids)-1):
        if ids[i] == ids[i+1]:
            total += 1
            if ids[i] in repet.keys():
                repet[ids[i]] += 1
            else:
                repet[ids[i]] = 1
    return repet

def meta_group_sys(group_des):
    messages = [{
        'role': 'system',
        'content': f"""
## Function
You are required to give a list of grades for the input entry to support the classification of the input entry.

## Group Traits
{group_des}

## Requirement
1. You should give the grade according to the relation between input entry and group trait.
2. The grade is between [0, 1], bigger grade means closer relation.

## Output Format
Please output in JSON format with only a list in it.
Example output: 
```json
    [x, x, x, x, ...]
```
"""
    }]
    return messages

async def group_from_proto(llm, agents, T, target_ids:list[int], group_des, log:bool=False) -> dict:
    groups = {}
    for id_ in target_ids:
        messages_meta_sys = meta_group_sys(group_des)
        group_metas = copy.deepcopy(list(group_des.keys()))
        for key in group_metas:
            if key not in groups.keys():
                groups[key] = []
        profile = get_profile_message(agents[id_])
        if log:
            print(profile)
        messages = messages_meta_sys + [{'role': 'user', 'content': f"""
## Input Entry:
{profile}
"""}]
        while True:
            try:
                resp = await llm.atext_request(messages)
                if log:
                    print(resp)
                resp = resp.strip('```json').strip('```')
                score = json.loads(resp)
                max_value = max(score)
                max_index = score.index(max_value)
                break
            except:
                pass
        if max_value >= T:
            try:
                name_ = group_metas[max_index]
            except:
                name_ = random.choice(group_metas)
            groups[name_].append(id_)
        else:
            while True:
                try:
                    messages.append({
                        'role': 'assistant', 'content': resp
                    })
                    messages.append({
                        'role': 'user', 'content':"""
        ## Function
        According to your analysis, the possibility of this entry belonging to those above groups is not high, please give some insights.
        
        ## Output Format
        JSON format of dictionay, which contains 2 keys ['group_name', 'trait']. group_name is the name of this new group. 'trait' is a summary of this group.
        
        Please give your answer:
        """
                    })
                    resp = await llm.atext_request(messages)
                    if log:
                        print(resp)
                    resp = resp.strip('```json').strip('```')
                    resp = json.loads(resp)
                    new_group_name = resp['group_name']
                    new_group_des = resp['trait']
                    break
                except:
                    pass
            # update
            groups[new_group_name] = [id_]
            group_des[new_group_name] = new_group_des
    return groups

async def repair(llm, agents, groups, noa, T, group_des, log:bool=False) -> dict:
    # delete redundant
    repet = check_repetation(groups)
    new_groups = copy.deepcopy(groups)
    for aid, time in repet.items():
        time_ = time
        for key, agent_ids in groups.items():
            agents_ = copy.deepcopy(agent_ids)
            for i, num in enumerate(agent_ids):
                if num == aid and time_ > 0:
                    agents_.pop(i)
                    time_ -= 1
            if time_ <= 0:
                new_groups[key] = agents_
                break
    
    # check missing
    miss = check_missing(new_groups, noa)

    to_add = await group_from_proto(llm, agents, T, miss, group_des)
    for key, _ in to_add.items():
        if key in new_groups.keys():
            new_groups[key] += copy.deepcopy(to_add[key])
        else:
            new_groups[key] = copy.deepcopy(to_add[key])
    return new_groups, group_des
    

async def profile_meta_group(llm, agents, IG:int, MAN:int, T:float, batch_size:int=100, log:bool=False) -> dict:
    """
    llm: a llm agent used for inference
    agents: agents used for meta learning
    IG: initial group
    MAN: meta agent number that used for initial meta learning
    T: threshold

    path: 
    1. [initial agents] -> LLM -> [meta group](IG)
    2. new agent -> LLM + [meta group] -> grade
    3.1 grade > threashold: add to the group
    3.2 grade < threashold: create a new group and add this group to [meta group]
    4. back to 2 until all agents are grouped
    """
    # * para check
    print("---Check parameters...")
    if T <= 0 or T >=1:
        print("Error: the threshold need to be set between (0, 1)")
        return None
    # length of total agents
    total_agent = len(agents)
    if IG >= total_agent:
        print("Error: the IG need to be set between [1, len(agents)]")
        return None
    if MAN > total_agent:
        print("Wrang: MAN greater than len(agents), set MAN to len(agents)")
        MAN = total_agent
    print("---Start Meta Learning...")
    # * step 1
    profile_list = """"""
    index = 0
    for agent in agents[:MAN]:
        profile = get_profile_message(agent)
        profile_list += f"{index}. {profile}\n"
        index += 1
    if log:
        print(profile_list)
    messages = [{
        'role': 'user',
        'content': f"""
## Function
According to your understanding, divide {MAN} enties groups. Each entry can be represents a personal information.

## Requirement
1. Totally {IG} groups.
2. Grouping should be based on maximizing differences in daily behavior between groups.
3. The grouping should be as general as possible to distinguish between different groups.
4. Each group has a descriptive name.
5. Each entry can only be assigned to the one and only group.

## Output Format
The output is required to be a dictionary in JSON format, containing 2 keys.
The first key is 'group', which is a dictonary. It contains {IG} inner keys, representing the group name, and the inner value is a list which contains all ids(int) belonging to the group.
The second key is 'trait', which  is a dictionry. It contains {IG} inner keys, representing the group name, and the inner value is a summary of the characteristics of the group.
Do not include any explanatory information in the output.

## Input
{profile_list}

Please give your answer:
"""
    }]
    resp = await llm.atext_request(messages)
    messages.append({'role': 'assistant', 'content': 'resp'})
    if log:
        print(resp)
    resp = resp.strip('```json').strip('```')
    intial_group = json.loads(resp)
    groups = intial_group['group']
    group_des = intial_group['trait']
    sum_agent = 0
    for key, value in groups.items():
        sum_agent += len(value)

    # * step 2: iteration
    messages_meta_sys = meta_group_sys(group_des)
    current_agent_index = index
    batch_groups = []
    group = []
    for i in range(index, len(agents)):
        group.append(i)
        if len(group) == batch_size or i == len(agents)-1:
            batch_groups.append(group)
            group = []
    tasks = [group_from_proto(llm, agents, T, batch_groups[i], group_des) for i in range(len(batch_groups))]
    results = await asyncio.gather(*tasks)
    # gather
    for group in results:
        for key, _ in group.items():
            if key in groups.keys():
                groups[key] += copy.deepcopy(group[key])
            else:
                groups[key] = copy.deepcopy(group[key])
                
    grouped_agent = 0
    for _, value in groups.items():
        grouped_agent += len(value)
    print(f"---Finish Learning, total input agents【{len(agents)}】, grouped agent 【{grouped_agent}】 in 【{len(list(groups.keys()))}】 categories...")
    return groups, group_des

In [ ]:
"""
Start in-context learning
arg_1: LLM client
arg_2: agents
arg_3: number of groups to create from the inital group of agents
arg_4: number of agents in initial group
arg_5: threshold
arg_6: acceleration option, we use async to accelerate the prototype learning process
arg_7: weather output log messages
"""
groups, group_des = await profile_meta_group(ag.soul, ag.agents, 5, 20, 0.5, batch_size=100, log=True)

In [8]:
"""you can store your learning results for accelerating the simulation process, please refer to the prototype dirctory to check out the format"""